# Resturant Reviews


***DataDescription***
> The data consists of 2 columns Review and Rating
> 
> Review: The reviews on Hotel were in the Review Column
> 
> Rating: The Ratings are denoted in the Rating column in the form of 1-5 
> 
> 1- Bad Review
> 
> 2- Average Review
>
> 3- Better Review
> 
> 4- Good Review
>
> 5- Best Review

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Adding Basic Liberaries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Loading the Data

In [ ]:
df = pd.read_csv('/kaggle/input/hotel-reviews/Hotel_Reviews.csv')
df.head()

In [ ]:
# Getting the shape of data
df.shape

* **Setting Parameters**

In [ ]:
vocab_size = 600
embedding_dim = 20
max_length = 1000
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20451

***Seperating data column to sentences and Labels***

In [ ]:
sentences = df['Review'].tolist()
labels  = df['Rating'].tolist()

# Getting Training and Testing Data

In [ ]:
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

# Setting Tokenizer And Padding data

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

***Converting data into arrays***

In [ ]:
import numpy as np
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

# Creating CNN model
# Adding Layers
# Compiling Models

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(42, activation='relu'),
    tf.keras.layers.Dense(6, activation='sigmoid')
])
loss_fn = keras.losses.SparseCategoricalCrossentropy()
model.compile(loss=loss_fn,optimizer='adam',metrics=['accuracy'])

In [ ]:
# Getting Summary
model.summary()

# Fiting CNN Model

In [ ]:
num_epochs = 50
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels))

# Plotting accuracy and loss Graph

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

* The 1st Graph Show the Difference B/w Increase in accuracy and val_accuracy 
* The 2nd Graph show the difference b/w decrease in loss and val_loss

***Decoding Sentences*** 

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_sentence(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_sentence(training_padded[0]))
print(training_sentences[0])
print(labels[0])

# Prediction on Testing Data

In [ ]:
for n in range(10):
    print(testing_sentences[n],': ',testing_labels[n],'\n')

> As we can see here the testing was all perfect!!!!!

> 1- Bad Review
> 
> 2- Average Review
>
> 3- Better Review
> 
> 4- Good Review
>
> 5- Best Review

# Getting Prediction with Randomly Created Reviews

In [ ]:
# Checking Predictions
sentence = ["Feeling as though you are on top of the world, immerse yourself in the beauty that this unique landscape has to offer, bordered by pristine white beaches and the brilliant blues of the sea. Feel inspired by the hopes and dreams that have"]
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(model.predict(padded))

* As we can see the sentences i created randomly were Predicted almost perfectly
* **The reviews** was **good** to it got Rating which is **almost equals** to **5**

**This Model's Accuracy is 67.50%**

***Please Leave your Valuable feedback in the comments below!!!!!!***